In [10]:
import pandas as pd
from scipy import spatial
from util import to_Cartesian,geo_to_cartesian,distToKM,kmToDIST
import geopandas as gpd
import numpy as np
from geopandas.tools import sjoin
import folium
from folium.plugins import MarkerCluster
import shapely
from shapely.geometry import Point
import unicodedata
import pysal as ps
from shapely import geometry
import sys
from folium import plugins
import string
import folium
from shapely import wkt, geometry
import json
from pprint import pprint
from openrouteservice import client, places
from openrouteservice import distance_matrix
from geopandas import GeoDataFrame
from shapely.geometry import Point
import osrm
from openrouteservice import geocoding
from algorithms import multi_objective_dijkstra

import re


#import osrm
#reload(sys)
#sys.setdefaultencoding('utf-8')

#Setup open-route service client
api_key = '58d904a497c67e00015b45fc07893889d08b4b6abcfc82938c195ec0'
clnt = client.Client(key=api_key)


# Make a new RequestConfig object
MyConfig = osrm.RequestConfig()
MyConfig.host = "http://router.project-osrm.org"  # Only change the host
osrm.RequestConfig.host = "router.project-osrm.org"
osrm.RequestConfig.host

#Read data
column_names = 'id;'.split(";")

all_data = pd.read_csv('./data/processed_data.csv',sep=",")

#Convert geo-coordinates to cartesian
all_data['x'], all_data['y'], all_data['z'] = geo_to_cartesian(all_data['long_wgs84'],all_data['lat_wgs84'])

stationArr = all_data[['lat_wgs84', 'long_wgs84']].as_matrix()

import geocoder

#https://geopy.readthedocs.io/en/stable/
from geopy.geocoders import Nominatim

addresses=[]
#Reverse geo code the address name
for i in range(len(all_data)):
    x = all_data['lat_wgs84'][i]
    y = all_data['long_wgs84'][i]
    address = all_data['address'][i]
    name = address.split('-')
    addresses.append(name[0].capitalize())
    
all_data['name'] = addresses
all_data.to_csv('./data/final_dataset.csv')
#locationlist = zip(all_data['long_wgs84'],all_data['lat_wgs84'])
locationlist = [[all_data['long_wgs84'][i],all_data['lat_wgs84'][i]] for i in range(len(all_data)) ]

In [2]:
#CREATE MAP
from folium.plugins import MarkerCluster
from folium import IFrame

#Create SF basemap specifying map center, zoom level, and using the default OpenStreetMap tiles
map = folium.Map(location=[50.821658, 4.394886], zoom_start=15)


points=[]

#Folium bugs when using numpy -> use a list of lists as an alternative
stationArr = [[all_data['lat_wgs84'][i],all_data['long_wgs84'][i],all_data['pm2.5'][i]] for i in range(len(all_data['latitude']))]
locationlist = [[all_data['lat_wgs84'][i],all_data['long_wgs84'][i]] for i in range(len(all_data))]


#Create empty lists to contain the point coordinates and the point pop-up information
coords, popups = [], [] 

map.save('routing.html')



In [3]:
for point in range(0, len(all_data)):
    #t='{0},({1}): {2}'.format(area_of_circle, points, estimate(radius, points))
    address=all_data['address'][point].split('-')[0].decode('latin-1').capitalize()
    x=all_data['latitude'][point]
    y=all_data['longitude'][point]
    print(address,x,y)
    pm=all_data['pm2.5'][point]
    text = u"<strong>{0}</strong><br>Lat: {1:.3f}<br>Long: {2:.3f}".format(address, x, y)
    popup = folium.Popup(text, parse_html=True)
    coords.append([x,y])
    popups.append(IFrame(text, width = 300, height = 100))
        


#Create a Folium feature group for this layer, since we will be displaying multiple layers
pt_lyr = folium.FeatureGroup(name = 'pt_lyr')

#Add the clustered points of crime locations and popups to this layer
pt_lyr.add_children(MarkerCluster(locations = coords, popups = popups))

#Add this point layer to the map object
map.add_children(pt_lyr)  




max_amount = float(all_data['pm2.5'].max())
map.add_child(plugins.HeatMap(stationArr,
                                 radius=15,
                                 min_opacity=0.2,
                                 max_val=max_amount,
                                 blur=15, 
                                 max_zoom=10))

map.save('routing.html')

(u'Ferm\xe9e', 50.812488999999999, 4.3792029999999995)
(u'Biestebroeck / biestebroek ', 50.833424000000001, 4.3133780000000002)
(u'Ten reuken ', 50.804454999999997, 4.4263690000000002)
(u'Hall des sports ', 50.877037000000001, 4.3008470000000001)
(u'Hotel president/president hotel ', 50.863875, 4.358708)
(u'Abbaye de la cambre/abdij ter kameren ', 50.820042000000001, 4.3747850000000001)
(u'Forest national / vorst nationaal ', 50.810085000000001, 4.3248639999999998)
(u'Place st jean/sint', 50.845123999999998, 4.3514860000000004)
(u'Vanderkindere ', 50.81418, 4.3477209999999999)
(u'Atomium ', 50.895485000000001, 4.3413110000000001)
(u'Altitude cent / hoogte honderd ', 50.816524999999999, 4.3362120000000006)
(u'Reine fabiola / koningin fabiola ', 50.872475000000001, 4.3142659999999999)
(u'Chee de waterloo/waterloosesteenweg 1050', 50.798988000000001, 4.3755809999999995)
(u'Tractebel ', 50.856582000000003, 4.4309880000000001)
(u'Place bockstael/bockstaelplaats ', 50.878079999999997, 4.3479

(u'Schuman ', 50.843359, 4.3799460000000003)
(u'Notre dame / onze lieve vrouw ', 50.870983000000003, 4.3975059999999999)
(u'Boileau ', 50.831679999999999, 4.4047689999999999)
(u'Clovis ', 50.850619999999999, 4.3811519999999993)
(u'Ulb plaine ', 50.818253000000006, 4.3957489999999995)
(u'Anneessens ', 50.844045000000001, 4.3450480000000002)
(u'Houffalize ', 50.865217999999999, 4.3777349999999995)
(u'Delacroix ', 50.845928000000001, 4.3238120000000002)
(u'Bois de la cambre ', 50.812179999999998, 4.3683209999999999)
(u'Masui ', 50.869296999999996, 4.3625699999999998)
(u'Genot ', 50.861783000000003, 4.3026070000000001)
(u'Square albert i / albert i', 50.838733000000005, 4.3217600000000003)
(u'Meiser ', 50.854653999999996, 4.3984510000000006)
(u'Louis titz ', 50.835723000000002, 4.3943949999999994)
(u'Tamaris ', 50.854824000000001, 4.306209)
(u'Fabry ', 50.837288000000001, 4.4346199999999998)
(u'Scheut ', 50.847648999999997, 4.3077139999999998)
(u'Maelbeek/maalbeek ', 50.843835999999996, 4.

(u'Hansen', 50.831078000000005, 4.3985459999999996)
(u'Sainte marie/sinte', 50.857779999999998, 4.368144)
(u'Zaman forest national/zaman vorst nationaal ', 50.813586999999998, 4.324681)
(u'Centre culturel / cultureel centrum ', 50.816947999999996, 4.4265509999999999)
(u'Avenue paul hymanslaan', 50.843617000000002, 4.4253239999999998)
(u'Jules cesar ', 50.834705, 4.4216430000000004)
(u'Ferm\xe9e ', 50.797739, 4.3360830000000004)
(u'Jambline de meux ', 50.847158, 4.3932549999999999)
(u'Claessens ', 50.873084999999996, 4.3585900000000004)
(u'Midi fonsny/zuid fonsny ', 50.835698999999998, 4.3388260000000001)
(u'Ferm\xe9e ', 50.875209999999996, 4.3179540000000003)
(u'Beekant ', 50.854165999999999, 4.3229480000000002)
(u'Ribaucourt ', 50.860861999999997, 4.3391820000000001)
(u'Erasme ', 50.814820000000005, 4.2653400000000001)
(u'Kennis ', 50.867643000000001, 4.3858169999999994)
(u'Saint guidon / sint guido ', 50.834755000000001, 4.3059620000000001)
(u'Porte de namur/naamsepoort ', 50.8390240

/home/god/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py:19: FutureWarning: Method `add_children` is deprecated. Please use `add_child` instead.
/home/god/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py:22: FutureWarning: Method `add_children` is deprecated. Please use `add_child` instead.


In [4]:
#CREATE GRAPH BETWEEN STATION
from graph import Graph
graph = Graph()
#convert the 30 km to cartesian coordinates distance
#dist = kmToDIST(1)

# create the KD-tree using the 3D cartesian coordinates
coordinates = list(zip(all_data['x'], all_data['y'], all_data['z']))
tree = spatial.cKDTree(coordinates)
# convert the 30 km to cartesian coordinates distance

# get all the points within 30 km from the reference point
#dist = kmToDIST(3)
#ix = tree.query_ball_point((x_ref, y_ref, z_ref), dist)
# get all the points within 30 km from the reference point
#ix = tree.query_ball_point((x_ref, y_ref, z_ref), dist)

all_data_array = all_data.as_matrix()
coord = all_data[['latitude','longitude']].as_matrix()
#coord = all_data[['long_wgs84','lat_wgs84']].as_matrix()
nodes=[]
#graph.add_nodes([i for i in range(len(all_data))])
for source in range(len(all_data)):
    #VISUALIZATION ON MAP
    address=all_data['address'][source].split('-')[0].decode('latin-1').capitalize()
    x=all_data['lat_wgs84'][source]
    y=all_data['long_wgs84'][source]
    pm=all_data['pm2.5'][source]
    name = all_data['name'][source]
    #print("Node : ",source,x,y,address)
    x_ref, y_ref, z_ref = to_Cartesian(y,x)
    # get the cartesian distances from the 10 closest points
    dist, ix = tree.query((x_ref, y_ref, z_ref), 5)
    nodes.append(i)
    graph.add_node(i,[x,y])
    #Add neighbors to graph
    for target in range(1,len(ix)):
        index = ix[target]
        distance = dist[target]
        x_target = coord[index][0]
        y_target = coord[index][1]
        pm_target=all_data['pm2.5'][index]
        #print("source",source,"index",index)
        graph.add_edge(source,index,distance,pm_target)
        #folium.PolyLine([[x, y], [x_target, y_target]]).add_to(map)
        #print("target",[x,y], [x_target, y_target])


        
print('done')
map

map.save('routing_final.html')




done


In [7]:
#etterbeek -> jaargetijden ->  petite suise -> ulb
#etterbeek -> Cimetiere d'ixelles -> ulb
#etterbeek -> petit suide -> ulb

ulb = [50.813724000000001,4.3842050000000006]

north = [50.860652000000002, 4.3581160000000008]
etterbeek = [50.822206999999999, 4.3893250000000004]

initial = graph.find_node(north)

final = graph.find_node(etterbeek)


noord = 27

vub = 39
etterbeek = 190
jaargetijden=263
petite_suide=268
cemitiere=109
ulb = 309
print(noord,etterbeek)
#multi_objective_dijkstra(graph,0,5)
print(graph.nodes[etterbeek].coordinates)
etterbeek_coord = graph.nodes[etterbeek].coordinates
jaargetijden_coord = graph.nodes[jaargetijden].coordinates
petite_suide_coord = graph.nodes[petite_suide].coordinates
cemitiere_coord = graph.nodes[cemitiere].coordinates
ulb_coord = graph.nodes[ulb].coordinates

from osrm import Point, simple_route

p1 = Point(latitude=2.386459, longitude=48.512369)

p2 = Point(latitude=2.536974, longitude=48.793416)

result = simple_route(p1, p2)

(27, 190)
[50.822206999999999, 4.3893250000000004]


HTTPError: HTTP Error 429: Unknown

In [6]:
#etterbeek -> jaargetijden ->  petite suise -> ulb
#etterbeek -> Cimetiere d'ixelles -> ulb
#etterbeek -> petit suide -> ulb

folium.PolyLine([etterbeek_coord,jaargetijden_coord]).add_to(map)
folium.PolyLine([jaargetijden_coord,petite_suide_coord]).add_to(map)
folium.PolyLine([petite_suide_coord,ulb_coord]).add_to(map)
folium.PolyLine([etterbeek_coord,cemitiere_coord]).add_to(map)
folium.PolyLine([cemitiere_coord,ulb_coord]).add_to(map)
folium.PolyLine([etterbeek_coord,petite_suide_coord]).add_to(map)
folium.PolyLine([petite_suide_coord,ulb_coord]).add_to(map)
map.save('routes_final.html')  
print(etterbeek_coord)

coordinates_distance_matrix = [etterbeek_coord,jaargetijden_coord,petite_suide_coord,ulb_coord,cemitiere_coord]
coordinates_distance_matrix = [[coord[1],coord[0]] for coord in coordinates_distance_matrix]
print(coordinates_distance_matrix)
map

[50.822206999999999, 4.3893250000000004]
[[4.3893250000000004, 50.822206999999999], [4.3853580000000001, 50.820391000000001], [4.3849710000000002, 50.817774999999997], [4.3842050000000006, 50.813724000000001], [4.391006, 50.816029]]


In [4]:
from algorithms import backpropagateroutes
#Create SF basemap specifying map center, zoom level, and using the default OpenStreetMap tiles
#map2 = folium.Map(location=[50.821658, 4.394886], zoom_start=15)


print(len(graph.nodes))

routes = backpropagateroutes(graph, etterbeek,ulb)
print(routes)
for route in routes:
    print("route",route)
    for i in range(1,len(route)):
        x_target , y_target = graph.get_node(route[i]).coordinates      
        #folium.PolyLine([[x, y], [x_target, y_target]]).add_to(map)
        #folium.PolyLine([[y, x], [y_target, x_target]]).add_to(map)

        #print(x,y)
        #address="test"
        #text = u"<strong>{0}</strong><br>Lat: {1:.3f}<br>Long: {2:.3f}".format(address, y, x)
        #popup = folium.Popup(text, parse_html=True)
        #folium.CircleMarker(locationlist[point],popup='test').add_to(map)
        #folium.Marker([x,y],popup=popup).add_to(map2)
        #folium.Marker([x_target,y_target],popup=popup).add_to(map2)
        x , y = [x_target,y_target]

        #Append lat and long coordinates to "coords" list
        #coords.append([x,y])
        #print(x,y,x_target,y_target)
    
    
map
map.save('map2.html')    

348


NameError: name 'etterbeek' is not defined

In [7]:
from algorithms import backpropagateroutes
#Create SF basemap specifying map center, zoom level, and using the default OpenStreetMap tiles
map2 = folium.Map(location=[50.821658, 4.394886], zoom_start=15)

#routes = backpropagateroutes(g, 0, 5)
x , y = ulb
text = u"<strong>{0}</strong><br>Lat: {1:.3f}<br>Long: {2:.3f}".format(address, x, y)
popup = folium.Popup(text, parse_html=True)
folium.Marker([x,y],popup=popup).add_to(map2)

x , y = graph.get_node(routes[0][2]).coordinates
x , y = vub_triomphe
text = u"<strong>{0}</strong><br>Lat: {1:.3f}<br>Long: {2:.3f}".format(address, x, y)
popup = folium.Popup(text, parse_html=True)
folium.Marker([x,y],popup=popup).add_to(map2)
print(routes)
print("initial",final)
for route in routes:
    print(route)
    for i in range(0,len(route)):
        x_target , y_target = graph.get_node(route[i]).coordinates      
        folium.PolyLine([[x, y], [x_target, y_target]]).add_to(map2)
        #folium.PolyLine([[y, x], [y_target, x_target]]).add_to(map)

        #address="test"
        #text = u"<strong>{0}</strong><br>Lat: {1:.3f}<br>Long: {2:.3f}".format(address, y, x)
        #popup = folium.Popup(text, parse_html=True)
        #folium.CircleMarker(locationlist[point],popup='test').add_to(map)
        #folium.Marker([x,y],popup=popup).add_to(map2)
        #folium.Marker([x_target,y_target],popup=popup).add_to(map2)
        x , y = [x_target,y_target]

        #Append lat and long coordinates to "coords" list
        coords.append([x,y])
        #print(x,y,x_target,y_target)

    
map2
map2.save('map2.html')  

TypeError: 'int' object is not iterable

In [18]:
#print(weights.)

folium.PolyLine([[50.840499999999999, 4.356878], [50.875826000000004,  4.32992476564]], color="red", weight=5, opacity=0.8).add_to(map)
folium.PolyLine([[50.821658, 4.394886], [50.81563, 4.9]], color="red", weight=5, opacity=0.8).add_to(map)


map
map.save('routing6.html')   

In [14]:

def getDistance(from_id, to_id):
    return int(pubs_matrix['durations'][from_id][to_id])


473


In [16]:
from openrouteservice import distance_matrix


#Create distance matrix
request = {'locations': coordinates_distance_matrix,
           'profile': 'driving-car',
           'metrics': ['distance']}
    
distance_matrix = clnt.distance_matrix(**request)
#print("Calculated {}x{} routes.".format(len(pubs_matrix['durations']),len(pubs_matrix['durations'][0])))


print(distance_matrix['distances'][0][1])

print(all_data.loc[0]['address'])

1166.75
FERM�E-TRAVAUX STIB - ULB SOLBOSCH / VUB SOLBOSCH - AV. F.ROOSEVELT/F.ROOSEVELTLAAN 19


In [17]:
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2
def getDistance(from_id, to_id):
    return int(distance_matrix['distances'][from_id][to_id])


tsp_size = len(coordinates_distance_matrix)
num_routes = 1
start = 0 # arbitrary start location
#coords_aoi = [(y,x) for x,y in aoi_coords] # swap (x,y) to (y,x)

optimal_coords = []
all_data
if tsp_size > 0:
    routing = pywrapcp.RoutingModel(tsp_size, num_routes, start)
    search_parameters = pywrapcp.RoutingModel.DefaultSearchParameters()

    # Create the distance callback, which takes two arguments (the from and to node indices)
    # and returns the distance between these nodes.
    dist_callback = getDistance
    routing.SetArcCostEvaluatorOfAllVehicles(dist_callback)
    # Solve, returns a solution if any.
    assignment = routing.SolveWithParameters(search_parameters)
    if assignment:
        # Total cost of the 'optimal' solution.
        print("Total duration: " + str(round(assignment.ObjectiveValue(), 3) / 60) + " minutes\n")
        index = routing.Start(start) # Index of the variable for the starting node.
        route = ''
        while not routing.IsEnd(index):
            
            for node in range(routing.nodes()):
                #all_data[routing.IndexToNode(index)]
                #print(all_data[node])
                print(all_data.loc[node]['address'])
                route += str(all_data[routing.IndexToNode(index)]) + ' -> '
            #index = assignment.Value(routing.NextVar(index))
        route += str(all_data[routing.IndexToNode(index)])
        #optimal_coords.append(pubs_coords[routing.IndexToNode(index)])
        print("Route:\n" + route)

Total duration: 54.95 minutes

FERM�E-TRAVAUX STIB - ULB SOLBOSCH / VUB SOLBOSCH - AV. F.ROOSEVELT/F.ROOSEVELTLAAN 19


KeyError: 0

In [ ]:
shapely.speedups.enable()




bounded_area = [[50.809956 ,4.375437],[50.809956 ,4.380227],[50.825817,4.404967],[50.825817 ,4.403449]]


#Sort dataframe by coordinates
#all_data = all_data.sort_values(['latitude', 'longitude'], ascending=[True, True])


#Reset the indexes of dataframe after sorting
#all_data = all_data.reset_index(drop=True)

#
#all_data = all_data[0:20]
locations = all_data[['lat_wgs84', 'long_wgs84']]
locations = all_data[['long_wgs84', 'lat_wgs84']]
locations = all_data[['longitude', 'latitude']]
locationlist = locations.values.tolist()


wkt_str = 'Polygon ((13.43926404 52.48961046, 13.42040115 52.49586382, 13.42541101 52.48808523, 13.42368155 52.48635829, 13.40788599 52.48886084, 13.40852944 52.487142, 13.40745989 52.48614988, 13.40439187 52.48499746, 13.40154731 52.48500125, 13.40038591 52.48373202, 13.39423818 52.4838664, 13.39425346 52.48577149, 13.38629096 52.48582648, 13.38626853 52.48486362, 13.3715694 52.48495055, 13.37402099 52.4851697, 13.37416365 52.48771105, 13.37353615 52.48798191, 13.37539925 52.489432, 13.37643416 52.49167597, 13.36821531 52.49333093, 13.36952826 52.49886974, 13.37360623 52.50416333, 13.37497726 52.50337776, 13.37764916 52.5079675, 13.37893813 52.50693045, 13.39923153 52.50807711, 13.40022883 52.50938108, 13.40443425 52.50777471, 13.4052848 52.50821063, 13.40802944 52.50618019, 13.40997081 52.50692569, 13.41152096 52.50489127, 13.41407284 52.50403794, 13.41490921 52.50491634, 13.41760145 52.50417013, 13.41943091 52.50564912, 13.4230412 52.50498109, 13.42720031 52.50566607, 13.42940229 52.50857222, 13.45335235 52.49752496, 13.45090795 52.49710803, 13.44765912 52.49472124, 13.44497623 52.49442276, 13.43926404 52.48961046))'

aoi_geom = wkt.loads(wkt_str) # load geometry from WKT string

aoi_coords = list(aoi_geom.exterior.coords) # get coords from exterior ring
aoi_coords = [(y,x) for x,y in aoi_coords] # swap (x,y) to (y,x). Really leaflet?!
aoi_centroid = aoi_geom.centroid # Kreuzberg center for map center

from openrouteservice import client, places




gdf = gpd.GeoDataFrame(all_data.drop(['latitude', 'longitude'], axis=1),
                                crs={'init': 'epsg:4326'},
                                geometry=[shapely.geometry.Point(xy) for xy in zip(all_data.latitude, all_data.longitude)])




#print(result)
from scipy import spatial


  
import math
from collections import namedtuple

def haversine_distance(origin, destination):
  """ Haversine formula to calculate the distance between two lat/long points on a sphere """

  radius = 6371 # FAA approved globe radius in km

  dlat = math.radians(destination.lat-origin.lat)
  dlon = math.radians(destination.lng-origin.lng)
  a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(origin.lat)) \
      * math.cos(math.radians(destination.lat)) * math.sin(dlon/2) * math.sin(dlon/2)
  c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
  d = radius * c

  # Return distance in km
  return int(math.floor(d))

LatLng = namedtuple('LatLng', 'lat, lng')


#for origin_coordinates in locationlist:
#    origin = LatLng(origin_coordinates[0], origin_coordinates[1]) # London
#    for destination_coordinates in locationlist:
#        destination = LatLng(destination_coordinates[0], destination_coordinates[1]) # Athens
     
# convert the grid points and reference points to cartesian coordinates

# convert the grid points and reference points to cartesian coordinates
#x, y, z = zip(*map(to_Cartesian, lats_1d, lons_1d))

all_data['x'], all_data['y'], all_data['z'] = zip(*map(to_Cartesian, all_data['lat_wgs84'], all_data['long_wgs84']))


In [ ]:
from math import *

def to_Cartesian(lat, lng):
    '''
    function to convert latitude and longitude to 3D cartesian coordinates
    '''
    R = 6371 # radius of the Earth in kilometers

    x = R * cos(lat) * cos(lng)
    y = R * cos(lat) * sin(lng)
    z = R * sin(lat)
    return x, y, z

def deg2rad(degree):
    '''
    function to convert degree to radian
    '''
    rad = degree * 2*np.pi / 360
    return(rad)

def rad2deg(rad):
    '''
    function to convert radian to degree
    '''
    degree = rad/2/np.pi * 360
    return(degree)

def distToKM(x):
    '''
    function to convert cartesian distance to real distance in km
    '''
    R = 6371 # earth radius
    gamma = 2*np.arcsin(deg2rad(x/(2*R))) # compute the angle of the isosceles triangle
    dist = 2*R*sin(gamma/2) # compute the side of the triangle
    return(dist)

def kmToDIST(x):
    '''
    function to convert real distance in km to cartesian distance 
    '''
    R = 6371 # earth radius
    gamma = 2*np.arcsin(x/2./R) 
    
    dist = 2*R*rad2deg(sin(gamma / 2.))
    return(dist)

In [ ]:
from math import cos, asin, sqrt

def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return 12742 * asin(sqrt(a))

def closest(data, v):
    return min(data, key=lambda p: distance(v['lat'],v['lon'],p['lat'],p['lon']))

tempDataList = [{'lat': 39.7612992, 'lon': -86.1519681}, 
                {'lat': 39.762241,  'lon': -86.158436 }, 
                {'lat': 39.7622292, 'lon': -86.1578917}]

v = {'lat': 39.7622290, 'lon': -86.1519750}
print(closest(tempDataList, v))

In [ ]:
for origin_coordinates in locationlist:
    origin = LatLng(origin_coordinates[0], origin_coordinates[1]) # London
    for destination_coordinates in locationlist:
        destination = LatLng(destination_coordinates[0], destination_coordinates[1]) # Athens
        result = osrm.simple_route(origin, destination)
 


In [ ]:
from scipy import spatial
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

plt.style.use('seaborn-poster')

# define map ranges
llat = 50.821658
ulat = 60
llon = 4.394886
ulon = 5

# generate grid points
lats = np.arange(llat, ulat, 0.1)
lons = np.arange(llon, ulon, 0.1)
lons, lats = np.meshgrid(lons, lats)

# flatten the locations
lons_1d = lons.flatten()
lats_1d = lats.flatten()

# reference point as the center of the map
lat_0 = (llat + ulat) / 2.
lon_0 = (llon + ulon) / 2.

coordinates = [[lons_1d[i],lats_1d[i]] for i in range(len(lats))]
print(coordinates)

map = folium.Map(location=[50.821658, 4.394886], zoom_start=15)





In [ ]:






#map.save('main.html')

In [ ]:


from scipy import spatial
tree = spatial.KDTree(locationlist)

for coordinates in locationlist:
    x = coordinates[0]
    y = coordinates[1]
    result = tree.query(coordinates)
    print(result)
#tree = spatial.KDTree(locationlist)
#tree.query([(21,21)])

In [ ]:
result = osrm.simple_route(
                      [21.0566163803209,42.004088575972], [20.9574645547597, 41.5286973392856],
                      output='route', overview="full", geometry='wkt')
print(result)

In [ ]:
locationlist = locationlist[0:10]

from openrouteservice import distance_matrix


request = {'locations': locationlist,
           'profile': 'driving-hgv',
           'metrics': ['duration']}
    
pubs_matrix = clnt.distance_matrix(**request)
print(pubs_matrix)
print("Calculated {}x{} routes.".format(len(pubs_matrix['durations']),len(pubs_matrix['durations'][0])))

In [ ]:
o,r, c = osrm.table(locationlist, output='dataframe')
print(o)

In [ ]:
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

def getDistance(from_id, to_id):
    return int(pubs_matrix['durations'][from_id][to_id])

tsp_size = len(locationlist)
num_routes = 10
start = 0 # arbitrary start location
coords_aoi = [(y,x) for x,y in aoi_coords] # swap (x,y) to (y,x)

optimal_coords = []

if tsp_size > 0:
    routing = pywrapcp.RoutingModel(tsp_size, num_routes, start)
    search_parameters = pywrapcp.RoutingModel.DefaultSearchParameters()

    # Create the distance callback, which takes two arguments (the from and to node indices)
    # and returns the distance between these nodes.
    dist_callback = getDistance
    routing.SetArcCostEvaluatorOfAllVehicles(dist_callback)
    # Solve, returns a solution if any.
    assignment = routing.SolveWithParameters(search_parameters)
    if assignment:
        # Total cost of the 'optimal' solution.
        print("Total duration: " + str(round(assignment.ObjectiveValue(), 3) / 60) + " minutes\n")
        index = routing.Start(start) # Index of the variable for the starting node.
        route = ''
#         while not routing.IsEnd(index):
        for node in range(routing.nodes()):
            optimal_coords.append(locationlist[routing.IndexToNode(index)])
            #route += str(pubs_addresses[routing.IndexToNode(index)]) + ' -> '
            index = assignment.Value(routing.NextVar(index))
        #route += str(pubs_addresses[routing.IndexToNode(index)])
        optimal_coords.append(locationlist[routing.IndexToNode(index)])
        print(optimal_coords)

In [ ]:
from openrouteservice import directions
import os.path

def style_function(color):
    return lambda feature: dict(color=color,
                              weight=3,
                              opacity=1)




folium.features.GeoJson(data=optimal_route,
                        name='Optimal Bar Crawl',
                        style_function=style_function('#6666ff'),
                       overlay=True).add_to(map)

map.add_child(folium.map.LayerControl())
map

In [ ]:
def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

df['dist'] = \
    haversine_np(df.LONG.shift(), df.LAT.shift(),
                 df.loc[1:, 'LONG'], df.loc[1:, 'LAT'])
    
    
    

## Data Prep
In this section I create two GeoDataFrames: one of crime points and one of census tract boundaries with crime densities. Both of these will then be plotted on a web map as separate layers.

### Read in Crime Data and Create a GeoDataFrame
First I read in a CSV file of San Francisco Police Incidents for the current year into a Pandas DataFrame. I downloaded the raw data from the San Francisco [Open Data Portal](https://data.sfgov.org/). Because there are so many crime incidents, I select a subset of the data: crimes in the "assault" category that were committed in the last 10 days. As shown below, this leaves me with 329 police incidents. 


In [ ]:
#Read in CSV file specifying date field and encoding. Sort by date
all_crime = pd.read_csv('SFPD_Incidents_-_Current_Year__2016_.csv', parse_dates = ['Date'],\
                        encoding = 'utf-8').sort_values('Date').reset_index(drop=True)

#Clean up the encoding on the Crime Description field 
all_crime.Descript = all_crime.Descript.apply(lambda x: unicodedata.normalize("NFKD", x))

#Create a field that contains a string representation of the date, for later use
all_crime['DateStr'] = all_crime.Date.apply(lambda x: x.strftime("%Y-%m-%d"))

#Identify those crimes that are categorized as assaults
is_assault = all_crime.Category == 'ASSAULT' 

#Identify those crimes that were committed in the most recent 10 days of the dataset
recent = all_crime.Date.isin(all_crime.Date.unique()[-10:]) 

#Subset the data to get assaults commited within the last 10 days
assaults = all_crime[is_assault & recent].drop_duplicates('IncidntNum').reset_index(drop = True)
assaults = assaults[['IncidntNum', 'Descript', 'DateStr', 'Time', 'Address','X', 'Y']]
print '{} assaults in the last 10 days'.format(str(len(assaults)))

I now want to convert the assault data Pandas DataFrame to a GeoPandas GeoDataFrame (a spatial version of the former). The raw crime data comes with lat/long coordinates, which I use these to create Shapely point geometry objects (these are the values in the "geometry" field for each record in a GeoDataFrame). I specify the spatial reference system as ESPG 4326 which represents the standard WGS84 coordinate system.

In [ ]:
#First create a GeoSeries of crime locations by converting coordinates to Shapely geometry objects
#Specify the coordinate system ESPG4326 which represents the standard WGS84 coordinate system
assault_geo = gpd.GeoSeries(assaults.apply(lambda z: Point(z['X'], z['Y']), 1),crs={'init': 'epsg:4326'})

#Create a geodataframe from the pandas dataframe and the geoseries of shapely geometry objects
assaults = gpd.GeoDataFrame(assaults.drop(['X', 'Y'], 1), geometry=assault_geo)
print assaults.head()

### Calculate Census Tract Crime Density
Next, I read in a Shapefile of San Francisco Census Tracts, which I also downloaded from the SF Open Data Portal. With GeoPandas, I can read a Shapefile directly into Python really easily. Then in one line of code, I spatially join census tracts to assaults (determine the census tract of each assault), and generate counts of assaults per census tract. Note that I use the ```to_crs``` function to convert assaults to the same coordinate system as Census Tracts (EPSG 3310) prior to spatially joining them.

Lastly, I calculate the number of assaults per square mile, which is the metric that I'm interested in plotting.

In [ ]:
#Read tracts shapefile into GeoDataFrame
tracts = gpd.read_file('sf_census_tracts.shp').set_index('CTFIPS10')

#Generate Counts of Assaults per Census Tract
#Spatially join census tracts to assaults (after projecting) and then group by Tract FIPS while counting the number of crimes
tract_counts = gpd.tools.sjoin(assaults.to_crs(tracts.crs), tracts.reset_index()).groupby('CTFIPS10').size()

#Calculate Assault Density, converting square meters to square miles.
tracts['AssaultsPSqMi'] = (tract_counts/(tracts.geometry.area*3.86102e-7)).fillna(0)
tracts = tracts.reset_index()
print tracts.head()

## Using Folium to Plot Data
The general approach I take here is to first create a Folium basemap and then add two layers to it: (1) a choropleth of census tracts, symbolized crime density, and (2) crime point locations. I write a separate function to plot each of these two layers, each of which takes a GeoDataFrame as its input. Folium takes unprojected lat/long coordinates for all of its plotting, so I make sure to convert all of my projected GeoDataFrames to WGS84 within the plotting functions.

###  Choropleth Layer of Tract Crime Density
As its inputs, my choropleth function takes a Folium map object, a GeoDataFrame, the name of the feature ID field, and the name of the field whose values will be symbolized. 

Leaflet uses GeoJSON objects to plot vector geometries (GeoJSON is a data format that is used to represent geographical features along with their non-spatial attributes). GeoPandas has a ```to_json``` method which I use to convert the GeoDataFrame to GeoJSON to be used as one of the inputs to Folium's choropleth function. I also specify the id field, which is used to link the geometry in the GeoJSON with the data in the GeoDataFrame.

The function also takes optional parameters for fill color, fill opacity, line opacity, number of classifiers, and classification scheme. All of these have default values if not specified. Folium / Leaflet uses the [Color Brewer](http://colorbrewer2.org/#type=sequential&scheme=YlOrRd&n=5) sequential color schemes, which can easily be specified to view different combinations. 

Lastly, I allow the user to specify the number of classes and the classification scheme. At this point, Folium has limited map classification options, so I instead use Pysal's choropleth [map classfication module](http://pysal.readthedocs.io/en/latest/library/esda/mapclassify.html) to provide some basic classification options. My function defaults to "Fisher_Jenks", but also has options for "Equal_Interval", and "Quantiles". 

Below, I first create a basemap that is centered in San Francisco, and then I run my function on this basemap specifying the Census Tract ID Field as well as the field I want to symbolize on (Assaults Per Square Mile).

In [ ]:
#Create SF basemap specifying map center, zoom level, and using the default OpenStreetMap tiles
crime_map = folium.Map([37.7556, -122.4399], zoom_start = 12)

def add_choropleth(mapobj, gdf, id_field, value_field, fill_color = 'YlOrRd', fill_opacity = 0.6, 
                    line_opacity = 0.2, num_classes = 5, classifier = 'Fisher_Jenks'):
    #Allow for 3 Pysal map classifiers to display data
    #Generate list of breakpoints using specified classification scheme. List of breakpoint will be input to choropleth function
    if classifier == 'Fisher_Jenks':
        threshold_scale=ps.esda.mapclassify.Fisher_Jenks(gdf[value_field], k = num_classes).bins.tolist()
    if classifier == 'Equal_Interval':
        threshold_scale=ps.esda.mapclassify.Equal_Interval(gdf[value_field], k = num_classes).bins.tolist()
    if classifier == 'Quantiles':
        threshold_scale=ps.esda.mapclassify.Quantiles(gdf[value_field], k = num_classes).bins.tolist()
    
    #Convert the GeoDataFrame to WGS84 coordinate reference system
    gdf_wgs84 = gdf.to_crs({'init': 'epsg:4326'})
    
    #Call Folium choropleth function, specifying the geometry as a the WGS84 dataframe converted to GeoJSON, the data as 
    #the GeoDataFrame, the columns as the user-specified id field and and value field.
    #key_on field refers to the id field within the GeoJSON string
    mapobj.choropleth(geo_str = gdf_wgs84.to_json(), data = gdf,
                columns = [id_field, value_field], key_on = 'feature.properties.{}'.format(id_field),
                fill_color = fill_color, fill_opacity = fill_opacity, line_opacity = line_opacity,  
                threshold_scale = threshold_scale)
    return mapobj

#Update basemap with choropleth
crime_map=add_choropleth(crime_map, tracts, 'CTFIPS10','AssaultsPSqMi')

### Create Crime Point Cluster Layer
Before displaying my map, I will also add the layer of crime point locations. Rather than display each individual point, I use Leaflet's marker clustering feature, which makes it easier to visualize large numbers of points by grouping together those that are close to each other. Additionally, I use popups to display information about a particular crime when the user clicks on a point. Folium lets you create HTML-rich popups called IFrames. I use this feature only in the most basic form, just to display a few lines of information: crime description, date, time, and address. There are obviously much more creative things that can be done with an IFrame popup (tables, graphs, sub-maps, etc.) but for my purposes this is all I need. 

My function takes as its inputs a Folium map object, a GeoDataFrame, and a list of fields to include in the popup. I run this function on my previously created map object (already updated with a choropleth layer), specifying 4 fields of interest that I want to display. 


In [ ]:
def add_point_clusters(mapobj, gdf, popup_field_list):
    #Create empty lists to contain the point coordinates and the point pop-up information
    coords, popups = [], [] 
    #Loop through each record in the GeoDataFrame
    for i, row in gdf.iterrows():
        #Append lat and long coordinates to "coords" list
        coords.append([row.geometry.y, row.geometry.x])
        #Create a string of HTML code used in the IFrame popup
        #Join together the fields in "popup_field_list" with a linebreak between them
        label = '<br>'.join([row[field] for field in popup_field_list])
        #Append an IFrame that uses the HTML string to the "popups" list 
        popups.append(IFrame(label, width = 300, height = 100))
        
    #Create a Folium feature group for this layer, since we will be displaying multiple layers
    pt_lyr = folium.FeatureGroup(name = 'pt_lyr')
    
    #Add the clustered points of crime locations and popups to this layer
    pt_lyr.add_children(MarkerCluster(locations = coords, popups = popups))
    
    #Add this point layer to the map object
    mapobj.add_children(pt_lyr)
    return mapobj

#Update choropleth with point clusters
crime_map = add_point_clusters(crime_map, assaults, ['Descript','Address','DateStr','Time'])

### Add Layer Control, Display and Save Map
As a finishing touch, I add Layer Control to the map, which allows me to toggle on/off either of my two layers (see widget on the top right). Then I save my finished map as an HTML and display it! 

I hope this was helpful in demonstrating some of the mapping capabilities of Leaflet accessed through the package Folium.  The functions I wrote provide a nice way of displaying two very common types of spatial data on a basemap and can obviously be tweaked to add more custom functionality.

In [ ]:
folium.LayerControl().add_to(crime_map) #Add layer control to toggle on/off
crime_map.save('sf_assaults.html') #save HTML
crime_map #display map